In [5]:
import pandas as pd
import json

fileDir = 'C:/Users/m/Documents/GitHub/population-data-map-overlay/data/info_by_zipcode/'
fileName = 'ACS_16_5YR_S1901_with_ann.csv'
outFileName = 'income-data.json'

#optionally specify a selection of column headers you want to keep
columnKeys = ['GEO.id2','HC01_EST_VC01','HC01_EST_VC02','HC01_EST_VC03','HC01_EST_VC04','HC01_EST_VC05','HC01_EST_VC06','HC01_EST_VC07','HC01_EST_VC08','HC01_EST_VC09','HC01_EST_VC10','HC01_EST_VC11','HC01_EST_VC13','HC01_EST_VC15','HC01_EST_VC18']


incomeByZipDf = pd.read_csv(fileDir+fileName,header=[0],skiprows=[1])
if columnKeys:
    incomeByZipDf = incomeByZipDf[columnKeys]

#generate a dictionary structure to map to json format
#use first column from input file (from columnKeys) as the key and everything else in another dictionary beneath that
#  where key=column_name, value=cell_val
jsonDict = {}
for rowIdx, rowData in incomeByZipDf.iterrows():
    zipId = str(rowData[columnKeys[0]]).rjust(5,'0')
    jsonDict[zipId] = dict(zip(columnKeys[1:],rowData[1:]))  #oh this is snazzy! it creates the dict under each zip in 1 line

#write to a json file. 
with open(fileDir+outFileName, 'w') as outFile:
    try:
        yourVarName
    except NameError:
        pass
    else:
        outFile.write('var '+yourVarName+'=\n')
    json.dump(jsonDict, outFile)
    

headerDf = pd.read_csv(fileDir+fileName,header=[0],nrows=1)
if columnKeys:
    headerDf = headerDf[columnKeys]
headerDict = dict(zip(headerDf.columns[1:], headerDf.iloc[0].values[1:]))

with open(fileDir+'descriptions_'+outFileName, 'w') as describeOutFile:
    json.dump(headerDict, describeOutFile)

In [11]:
incomeByZipDf[columnKeys[2:]].describe()  #why is ity only giving me these 2 columns?!? I'm so confused

,HC01_EST_VC02,HC01_EST_VC03,HC01_EST_VC04,HC01_EST_VC05,HC01_EST_VC06,HC01_EST_VC07,HC01_EST_VC08,HC01_EST_VC09,HC01_EST_VC10,HC01_EST_VC11,HC01_EST_VC13,HC01_EST_VC15,HC01_EST_VC18
count,33120,33120,33120,33120,33120,33120,33120,33120,33120,33120,33120,33120,33120
unique,518,419,537,511,560,625,505,503,323,494,19534,25951,794
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,-
freq,2985,3287,1758,1758,1389,1110,1885,2313,6195,7391,2008,586,580


In [6]:
otherFile = fileDir+'descriptions.txt'
with open(otherFile, 'w') as describeTxtFile:
    describeTxtFile.write(str(headerDict))

In [10]:
otherFile = fileDir+'descriptions2.txt'
with open(otherFile, 'w') as describeTxtFile:
    for key,val in headerDict.items():
        describeTxtFile.write('{}: {}'.format(key,val) +'\n')

In [ ]:
#below is copied from shp2gjson. maybe should make a module...
# *my state to zip mapping is incomplete. some smaller zips don't have their state listed here. They'll end up blank in leaflet

In [12]:
# reads/returns the pandas dataframe containing a list of state + zipcode pairs, either from file or from census beauru
# will 'cache' (locally save on disk) the result if read from census and use the local copy next time requested
def get_state_zip_df():
    saveFile = '../data/shapefiles-us_zips/zip_state_map.csv'
    # try to read read pre-downloaded and massaged file (if run/'cached' before), else download and save
    # feels sloppy
    try:
        stateZipDf = pd.read_csv(saveFile, dtype=str)
    except FileNotFoundError:
        zipCol = 'ZCTA5'
        stateCol = 'STUSAB'
        stateIdCol = 'STATE'
        readType = {zipCol: str, stateCol: str}

        zipSourceURL = 'https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_place_rel_10.txt'
        zipInfoDf = pd.read_csv(zipSourceURL, usecols=[zipCol, stateIdCol], dtype=readType)
        zipInfoDf.drop_duplicates(inplace=True)  # not sure what this data means, but there are several rows per zip

        stateSourceURL = 'https://www2.census.gov/geo/docs/reference/state.txt'
        stateInfoDf = pd.read_csv(stateSourceURL, sep='|')

        # pandas JOIN on state(FIPS ID)
        stateZipDf = pd.merge(zipInfoDf, stateInfoDf, left_on=stateIdCol, right_on=stateIdCol)
        stateZipDf = stateZipDf[[zipCol, stateCol]]
        #stateZipDf[zipCol] = stateZipDf[zipCol].apply(lambda x: str(x).rjust(5, '0'))  # convert zip from int to 5 char str #passing dtype=str on read already accomplishes this

        stateZipDf.to_csv(saveFile, index=False)  # save, don't print index column

    return stateZipDf

# pass the stateZipDf from get_state_zip_df()
# zip2StateDict[zipCode] = [state_zip_code_is_in, maybe_more_than_1_state]
# state2ZipDict[state][zipCode]=''
def gen_state_zip_dict(stateZipDf):
    zipCol = 'ZCTA5'
    stateCol = 'STUSAB'

    zip2StateDict = {}
    state2ZipDict = {}
    for row in stateZipDf.iterrows():
        zipCode = row[1][zipCol]
        stateStr = row[1][stateCol]

        if zipCode in zip2StateDict:
            # print(zipCode, stateStr)  # zip codes in more than 1 state. interesting
            zip2StateDict[zipCode].append(stateStr)
        else:
            zip2StateDict[zipCode] = [stateStr]

        if stateStr in state2ZipDict:
            state2ZipDict[stateStr][zipCode] = True
        else:
            state2ZipDict[stateStr] = {zipCode: True}

    return zip2StateDict, state2ZipDict

def create_state_zip_income_files(inputFile, targetBase):
    zipDf = get_state_zip_df()
    zipDict, stateDict = gen_state_zip_dict(zipDf)

    stateCnt = 0
    for state in stateDict.keys():
        print(stateCnt, ': ', state)
        stateCnt += 1
        sinkFile = targetFile+'_'+state+'.json'
        # gdal doesn't know how to overwrite a file
        if os.path.isfile(sinkFile):
            os.remove(sinkFile)

        with fiona.open(shapeFile+'.shp', 'r') as src:
            meta = src.meta
            meta['schema']['properties'] = {'zip': 'str'}
            meta['driver'] = 'GeoJSON'
            with fiona.open(sinkFile, 'w', **meta) as sink:
                for rec in src:
                    #only include zips in this state
                    if rec['properties']['ZCTA5CE10'] not in stateDict[state]:
                        continue

                    # I don't like displaying all the quirks of zip boundaries (and may be more costly to render), but this block isn't necessary
                    # not polygon means the multi-level/nested polygon thing. This removes those complicated compound shapes
                    if rec['geometry']['type'] != 'Polygon':
                        # simplify multipolygon to polygon:
                        # if polygon has negative space, the first element is the outer bound, skip other/interior pieces
                        # if polygon consists of multiple, separate pieces (like islands) we include all of those
                        newCoords = [coordSet[0] for coordSet in rec['geometry']['coordinates']]
                        rec['geometry']['coordinates'] = newCoords
                        rec['geometry']['type'] = 'Polygon'

                    rec['properties'] = {'zip': rec['properties']['ZCTA5CE10']}
                    sink.write(rec)

    return

create_state_zip_income_files('../data/info_by_zipcode/ACS_16_5YR_S1901_with_ann.csv', )

In [ ]:
fileDir = 'C:/Users/m/Documents/GitHub/population-data-map-overlay/data/info_by_zipcode/'
fileName = 'ACS_16_5YR_S1901_with_ann.csv'
outFileName = 'income-data.json'

#optionally specify a selection of column headers you want to keep
columnKeys = ['GEO.id2','HC01_EST_VC01','HC01_EST_VC02','HC01_EST_VC03','HC01_EST_VC04','HC01_EST_VC05','HC01_EST_VC06','HC01_EST_VC07','HC01_EST_VC08','HC01_EST_VC09','HC01_EST_VC10','HC01_EST_VC11','HC01_EST_VC13','HC01_EST_VC15','HC01_EST_VC18']


incomeByZipDf = pd.read_csv(fileDir+fileName,header=[0],skiprows=[1])
if columnKeys:
    incomeByZipDf = incomeByZipDf[columnKeys]

#generate a dictionary structure to map to json format
#use first column from input file (from columnKeys) as the key and everything else in another dictionary beneath that
#  where key=column_name, value=cell_val
jsonDict = {}
for rowIdx, rowData in incomeByZipDf.iterrows():
    zipId = str(rowData[columnKeys[0]]).rjust(5,'0')
    jsonDict[zipId] = dict(zip(columnKeys[1:],rowData[1:]))  #oh this is snazzy! it creates the dict under each zip in 1 line